In [2]:
# Constants
train_columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
test_columns = ['UserID', 'MovieID', 'Timestamp']
movies_columns = ['MovieID', 'Title', 'Genres']
users_columns = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']

In [41]:
import pandas as pd
import numpy as np
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise.model_selection import cross_validate

movies = pd.read_csv('dataset/movies.txt', sep='::', header=None, engine='python')
# movies.drop(0, inplace=True, axis=1)
movies.columns = movies_columns
users = pd.read_csv('dataset/users.txt', sep='::', header=None, engine='python')
users.columns = users_columns
training = pd.read_csv('dataset/training.txt', sep=',', header=None, engine='python')
training.columns = train_columns
testing = pd.read_csv('dataset/testing.txt', sep=',', header=None, engine='python')
testing.columns = test_columns

In [8]:
training.head()

,UserID,MovieID,Rating,Timestamp
0,63,3006,4,977899676
1,1125,10,2,975625890
2,181,2384,2,977088920
3,1968,1610,4,974742431
4,4156,2410,4,965342309


In [13]:
reader = Reader(), SV

In [14]:
data = Dataset.load_from_df(training[['UserID', 'MovieID', 'Rating']], reader)

In [42]:
svd = SVDpp()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8785  0.8690  0.8755  0.8751  0.8775  0.8725  0.8782  0.8719  0.8714  0.8712  0.8741  0.0032  
MAE (testset)     0.6862  0.6802  0.6844  0.6830  0.6853  0.6810  0.6845  0.6819  0.6793  0.6790  0.6825  0.0024  
Fit time          1850.44 1792.87 1743.43 1743.58 1741.25 1762.72 1778.32 1851.56 1848.07 1800.65 1791.29 43.01   
Test time         20.77   17.64   17.77   17.32   17.87   17.84   19.32   19.43   19.59   19.09   18.66   1.07    


{'test_rmse': array([0.87854386, 0.8690244 , 0.87554615, 0.8750723 , 0.87748652,
        0.87254289, 0.87817644, 0.87185267, 0.8714379 , 0.87123178]),
 'test_mae': array([0.68623264, 0.68024717, 0.68441503, 0.68298868, 0.6852983 ,
        0.68101199, 0.68450856, 0.68193111, 0.67930751, 0.67904149]),
 'fit_time': (1850.441700220108,
  1792.8698172569275,
  1743.4295904636383,
  1743.5844175815582,
  1741.245540857315,
  1762.7169616222382,
  1778.3157885074615,
  1851.5571711063385,
  1848.0735816955566,
  1800.6545011997223),
 'test_time': (20.76769495010376,
  17.63636064529419,
  17.76628589630127,
  17.322667360305786,
  17.866164207458496,
  17.835136890411377,
  19.319602966308594,
  19.43219256401062,
  19.591922998428345,
  19.088038206100464)}

In [57]:
from surprise.model_selection import GridSearchCV

svd = SVD()

param_grid = { 'lr_all': [0.002, 0.005, 0.1],
                'reg_all': [0.02, 0.4, 0.6]
            }

# best params?
# 'n_epochs': [100]
# n_factors: 150
# lr_all: 0.005
# reg_all: 0.02

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

# cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

0.9076932482433051
{'lr_all': 0.005, 'reg_all': 0.02}


In [52]:
# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

In [43]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [61]:
# svd.predict(4156, 2410, 4)
# algo.predict(4156, 2410, 4)
svd2 = SVDpp(n_factors= 150, 
n_epochs= 100, 
lr_all= 0.005,  # or 0.01?
reg_all= 0.1)

trainset = data.build_full_trainset()
svd2.fit(trainset)

In [62]:
predictions = []
for index, row in testing.iterrows():
    # print(row)
    predictions.append(svd2.predict(row['UserID'], row['MovieID']).est)
    # break
predictions

[2.9360119531802558,
 2.4777601030269643,
 3.3694951210298916,
 3.7249330507761447,
 4.170095945378945,
 3.781359616939779,
 3.82086151183379,
 3.1945207344624653,
 3.1385248378650776,
 2.2485023158582367,
 4.2415329911996675,
 3.966534480040029,
 3.3036920351200796,
 4.555370597204166,
 3.669896569579233,
 3.7031723264527776,
 3.815516151462493,
 2.9047982997385233,
 3.8712029881439003,
 3.599001522590009,
 3.627566490460992,
 4.380692092818712,
 4.246637387906745,
 3.6058311812438535,
 2.772726589274294,
 3.9407170056644185,
 3.3488520769020123,
 3.6284457215745616,
 3.785003240199682,
 3.5010680938018375,
 3.496526815031696,
 3.3133058019053654,
 3.9751620254662665,
 2.995024346064027,
 3.3732677251381507,
 4.0690566414277285,
 3.7578962666084004,
 3.7722237522307096,
 4.719978839009908,
 3.898450157671094,
 5,
 3.2929992655231834,
 3.891878853994906,
 4.649370115555865,
 4.142531946491355,
 2.8982016454876662,
 2.8542691684440196,
 3.0753111961568176,
 3.4410957780053084,
 3.927306

In [67]:
# predictions = svd.test()
result = pd.DataFrame(predictions, columns=['Predicted'])
result.index = result.index + 1
result.index.names = ['ID']
result.head()
result.to_csv('SVDtuned.csv')

In [49]:
result.head()

,Predicted
ID,
1,2.932375
2,2.308206
3,3.204498
4,3.665068
5,4.405108


In [66]:
result.to_csv('SVDtuned.csv')

In [ ]:
svd2

In [ ]:
svd2